In [1]:
import os 
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

2023-04-08 11:46:10.042724: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 11:46:11.387066: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train_path = "/home/kalyan/DataSets/animals/train/train/"
val_path = "/home/kalyan/DataSets/animals/test/test/"
train_df = pd.read_csv("/home/kalyan/DataSets/animals/train.csv")
test_df = pd.read_csv("/home/kalyan/DataSets/animals/test.csv")

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
import math

# Define constants
NUM_CLASSES = 30
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_df, validation_df = train_test_split(train_df, test_size=0.2, random_state=42)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
    
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_path,
    x_col="Image_id",
    y_col="Animal",
    batch_size=32,
    class_mode="categorical",
    target_size=(150, 150))

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=validation_df,
    directory=train_path,
    x_col="Image_id",
    y_col="Animal",
    batch_size=32,
    class_mode="categorical",
    target_size=(150, 150))


# Define learning rate scheduler
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 30:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.5
    print('Learning rate:', lr)
    return lr

# Define ResNet50 model with modified top layers
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer=Adam(lr=lr_schedule(0)),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
lr_scheduler = LearningRateScheduler(lr_schedule)
early_stop = EarlyStopping(monitor='val_loss', patience=5)


In [ ]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=len(train_generator),
                              epochs=EPOCHS,
                              validation_data=val_generator,
                              validation_steps=len(val_generator),
                              callbacks=[lr_scheduler, early_stop])

VGG IMPLEMENTATION

In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os

In [2]:
path = "/home/kalyan/DataSets/animals/"

In [3]:
print(os.listdir(path))

['vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 'train.csv', 'trainLabels.npy', 'trainbeg.npy', 'test.csv', 'sample_submission.csv', 'testbeg.npy', 'self_vgg16.h5', 'test', 'train']


In [4]:
X_train = np.load(path + "trainbeg.npy")

In [5]:
X_test = np.load(path + "testbeg.npy")

In [6]:
Y_train =  np.load(path + "trainLabels.npy")
print(Y_train.shape)
Y_train = Y_train.reshape(Y_train.shape[0])
np.squeeze(Y_train)
print(Y_train.shape)
print(Y_train)

(13000, 1)
(13000,)
[11 26 10 ... 26 26 26]


In [8]:
inp_dm = X_train.shape[1]
vgg_model_path = path + '/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
from keras.applications import VGG16
conv_base = VGG16(weights=vgg_model_path,include_top=False,input_shape=(inp_dm,inp_dm, 3))
conv_base.trainable = False

for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        layer.trainable = True
    if layer.name == 'block4_conv1':
        layer.trainable = True    
    else:
        layer.trainable = False

conv_base.summary()

2023-04-08 10:15:51.071541: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 10:15:51.705256: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-08 10:15:53.901629: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-08 10:15:53.920613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [9]:
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import Input, BatchNormalization
from keras.layers import Dense

def VGG16_classifier():    
    model = Sequential()
    model.add(conv_base)
    model.add(Flatten()) 
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(30, activation='softmax'))
    model.summary()
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [10]:
from keras.callbacks import  ReduceLROnPlateau
from keras.utils import to_categorical
model1 = VGG16_classifier()
callbacks_list = [ReduceLROnPlateau(monitor='loss',factor=0.2,patience=3)]

history_vgg = model1.fit(x = X_train/255.,y = to_categorical(Y_train, num_classes=30),batch_size=64,epochs=100,callbacks = callbacks_list, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 30)                15390     
                                                                 
Total params: 15,779,166
Trainable params: 1,064,478
Non-trainable params: 14,714,688
_________________________________________________________________


2023-04-08 10:21:39.215870: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 638976000 exceeds 10% of free system memory.
2023-04-08 10:21:39.634430: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 638976000 exceeds 10% of free system memory.


Epoch 1/100


2023-04-08 10:21:41.130758: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2023-04-08 10:21:43.301785: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x33347c60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-08 10:21:43.301827: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-04-08 10:21:43.307899: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-08 10:21:43.426714: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


204/204 [==============================] - 15s 53ms/step - loss: 2.5755 - accuracy: 0.2758 - lr: 0.0010
Epoch 2/100
204/204 [==============================] - 10s 50ms/step - loss: 2.1050 - accuracy: 0.3878 - lr: 0.0010
Epoch 3/100
204/204 [==============================] - 10s 51ms/step - loss: 1.8954 - accuracy: 0.4429 - lr: 0.0010
Epoch 4/100
204/204 [==============================] - 10s 50ms/step - loss: 1.7469 - accuracy: 0.4845 - lr: 0.0010
Epoch 5/100
204/204 [==============================] - 10s 50ms/step - loss: 1.6274 - accuracy: 0.5157 - lr: 0.0010
Epoch 6/100
204/204 [==============================] - 10s 48ms/step - loss: 1.5175 - accuracy: 0.5464 - lr: 0.0010
Epoch 7/100
204/204 [==============================] - 10s 50ms/step - loss: 1.4035 - accuracy: 0.5821 - lr: 0.0010
Epoch 8/100
204/204 [==============================] - 10s 50ms/step - loss: 1.3106 - accuracy: 0.6069 - lr: 0.0010
Epoch 9/100
204/204 [==============================] - 10s 51ms/step - loss: 1.2202 

In [30]:
model1.save(path + "self_vgg16.h5")

In [12]:
from sklearn.metrics import accuracy_score
y_train_predict = np.argmax(model1.predict(X_train/255.),axis=1)
print(accuracy_score(Y_train,y_train_predict))

2023-04-08 10:44:16.730940: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 638976000 exceeds 10% of free system memory.
2023-04-08 10:44:17.240159: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 638976000 exceeds 10% of free system memory.


407/407 [==============================] - 11s 25ms/step
0.9998461538461538


In [18]:
y_test_predict = model1.predict(X_test/255.)
print(y_test_predict)

188/188 [==============================] - 5s 25ms/step
[[1.49796442e-08 1.94970102e-08 1.63242646e-08 ... 3.26142690e-05
  8.16342393e-12 2.67197939e-12]
 [5.12042631e-09 3.37313804e-06 1.42101007e-05 ... 6.39694557e-03
  3.34410346e-04 1.75660261e-06]
 [9.92887437e-01 1.99221631e-05 1.50331267e-04 ... 4.59864680e-07
  8.47705394e-07 5.19748486e-04]
 ...
 [8.19700432e-08 1.92425460e-01 5.47298882e-03 ... 1.72403473e-02
  7.84731749e-03 6.47731051e-02]
 [3.77091638e-08 5.50205048e-10 2.93335400e-09 ... 2.63111175e-13
  1.70984968e-08 1.47328765e-05]
 [2.30001600e-08 5.84512634e-07 4.77698428e-04 ... 1.14654652e-09
  2.68664880e-04 2.85157676e-05]]


In [21]:
data_classes = ["antelope","bat","beaver","bobcat","buffalo","chihuahua","chimpanzee","collie","dalmatian","german+shepherd","grizzly+bear",
                "hippopotamus","horse","killer+whale","mole","moose","mouse","otter","ox","persian+cat","raccoon","rat","rhinoceros","seal",
                "siamese+cat","spider+monkey","squirrel","walrus","weasel","wolf"]
label_df = pd.DataFrame(data=y_test_predict, columns= data_classes)

label_df.head(10)

subm = pd.DataFrame()


te_label = pd.read_csv(path + '/test.csv')


print(te_label['Image_id'])

subm['image_id'] = te_label['Image_id']

#print(subm.head(10))
subm = pd.concat([subm, label_df], axis=1)

subm.to_csv('submitvgg.csv',index = False)

subm.head(10)

0          Img-1.jpg
1          Img-2.jpg
2          Img-3.jpg
3          Img-4.jpg
4          Img-5.jpg
            ...     
5995    Img-5996.jpg
5996    Img-5997.jpg
5997    Img-5998.jpg
5998    Img-5999.jpg
5999    Img-6000.jpg
Name: Image_id, Length: 6000, dtype: object


,image_id,antelope,bat,beaver,bobcat,buffalo,chihuahua,chimpanzee,collie,dalmatian,...,raccoon,rat,rhinoceros,seal,siamese+cat,spider+monkey,squirrel,walrus,weasel,wolf
0,Img-1.jpg,1.497964e-08,1.949701e-08,1.632426e-08,2.997081e-07,9.981319e-01,1.231509e-14,1.183655e-06,1.668450e-08,6.104327e-14,...,2.004127e-08,1.471354e-10,9.032629e-05,1.185133e-05,3.054609e-10,4.012723e-12,1.820806e-10,3.261427e-05,8.163424e-12,2.671979e-12
1,Img-2.jpg,5.120426e-09,3.373138e-06,1.421010e-05,3.276680e-06,3.929587e-08,2.162432e-03,2.851347e-05,1.032950e-09,3.383173e-03,...,3.258814e-06,4.289428e-03,3.653901e-02,6.892457e-01,2.293787e-05,6.809890e-05,1.472451e-04,6.396946e-03,3.344103e-04,1.756603e-06
2,Img-3.jpg,9.928874e-01,1.992216e-05,1.503313e-04,1.374127e-06,1.694746e-05,1.291535e-03,1.519165e-07,1.276257e-05,2.091474e-06,...,1.263892e-06,2.313075e-03,9.207237e-06,7.303223e-04,1.138934e-06,1.798488e-06,3.525436e-06,4.598647e-07,8.477054e-07,5.197485e-04
3,Img-4.jpg,1.369893e-10,1.904640e-04,3.816333e-09,1.398830e-06,3.481209e-18,9.695458e-01,3.086653e-06,7.569159e-08,1.470809e-04,...,4.924225e-09,1.224925e-02,3.389440e-08,6.654133e-09,7.598905e-03,6.611165e-07,1.814470e-06,6.837266e-08,6.549297e-05,4.232138e-11
4,Img-5.jpg,8.211344e-12,3.340067e-02,1.864996e-08,2.914349e-05,2.445964e-09,2.676726e-03,7.929195e-11,7.087166e-10,1.098022e-06,...,1.274798e-06,2.702370e-03,1.483843e-06,4.859417e-06,9.286709e-01,1.410121e-05,2.454261e-02,1.100330e-05,5.236225e-06,1.364662e-06
5,Img-6.jpg,7.508039e-13,2.558778e-09,7.590010e-07,2.435752e-08,3.482711e-16,2.412254e-10,9.686543e-08,1.789664e-13,4.716123e-10,...,3.393363e-08,4.761771e-09,1.263245e-11,9.999324e-01,6.322891e-11,1.635204e-06,6.982609e-09,5.605492e-05,1.138195e-06,4.842059e-08
6,Img-7.jpg,2.908742e-03,1.498656e-03,2.827088e-04,1.383671e-02,1.988077e-04,2.971498e-06,8.234620e-05,2.013518e-02,9.251644e-03,...,4.716255e-04,6.133314e-05,3.423745e-05,1.244897e-05,5.734227e-06,2.272476e-04,5.528277e-04,4.736444e-05,6.895407e-02,1.430517e-01
7,Img-8.jpg,1.097310e-09,5.041008e-08,1.817613e-10,2.732491e-08,5.924206e-10,6.332332e-06,3.992504e-07,6.417870e-05,2.947899e-13,...,6.921695e-08,3.402390e-07,4.490989e-10,5.041937e-07,2.049288e-07,9.910899e-08,2.662839e-10,1.715358e-09,9.265786e-08,4.991007e-09
8,Img-9.jpg,8.880340e-06,1.970157e-03,1.220477e-06,3.616779e-07,3.481840e-05,7.900029e-05,1.801909e-07,1.891657e-06,5.980168e-09,...,5.434130e-06,2.455263e-05,4.188533e-09,1.650384e-08,7.477143e-05,3.522885e-08,1.174824e-05,1.698952e-07,1.323324e-04,8.787772e-09
9,Img-10.jpg,1.121856e-09,8.563801e-11,1.642846e-08,2.129274e-06,2.055223e-17,2.936892e-03,5.171676e-08,5.284942e-02,9.349185e-01,...,1.474701e-04,1.552514e-08,3.966292e-11,3.660392e-11,1.117981e-10,1.237980e-08,1.042077e-06,2.002563e-13,9.657186e-09,3.109099e-05


Resnet Architecture

In [9]:
from keras.applications import ResNet50


In [8]:
from keras.models import Sequential, load_model, Model
from keras.layers import *
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

2023-04-08 11:47:56.196355: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 11:47:57.042376: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Dense
from keras import regularizers

# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Add a dense layer on top of the ResNet50 model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
predictions = Dense(30, activation='softmax')(x)

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

In [18]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 70, 70, 3)    0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 32, 32, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 32, 32, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

In [19]:
model.compile(optimizer=Adam(lr=0.0001),
                loss='categorical_crossentropy',        
                metrics=['accuracy'])


/home/kalyan/miniconda3/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [20]:
model.fit(X_train/255., to_categorical(Y_train, num_classes=30), batch_size=64, epochs=10, verbose=1)

2023-04-08 11:50:05.667573: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 638976000 exceeds 10% of free system memory.
2023-04-08 11:50:06.111693: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 638976000 exceeds 10% of free system memory.


Epoch 1/10


2023-04-08 11:50:11.224456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2023-04-08 11:50:13.184293: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 673.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-08 11:50:13.184344: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.34GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-08 11:50:13.503382: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 400.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains 

204/204 [==============================] - 36s 136ms/step - loss: 14.4476 - accuracy: 0.4005
Epoch 2/10
204/204 [==============================] - 27s 130ms/step - loss: 10.3265 - accuracy: 0.7527
Epoch 3/10
204/204 [==============================] - 27s 130ms/step - loss: 7.3759 - accuracy: 0.9195
Epoch 4/10
204/204 [==============================] - 27s 130ms/step - loss: 5.2589 - accuracy: 0.9655
Epoch 5/10
204/204 [==============================] - 27s 130ms/step - loss: 3.6922 - accuracy: 0.9768
Epoch 6/10
204/204 [==============================] - 27s 130ms/step - loss: 2.5419 - accuracy: 0.9807
Epoch 7/10
204/204 [==============================] - 27s 130ms/step - loss: 1.7838 - accuracy: 0.9665
Epoch 8/10
204/204 [==============================] - 27s 130ms/step - loss: 1.2482 - accuracy: 0.9609
Epoch 9/10
204/204 [==============================] - 27s 130ms/step - loss: 0.9171 - accuracy: 0.9548
Epoch 10/10
204/204 [==============================] - 27s 130ms/step - loss: 0.62

In [21]:
model.save(path + "self_resnet50.h5")

In [22]:
#predicting on test data
y_test_predict = model.predict(X_test/255.)
print(y_test_predict)


188/188 [==============================] - 6s 22ms/step
[[5.6558970e-06 4.6858077e-05 1.1881919e-02 ... 4.2412494e-04
  7.0931906e-06 1.4301834e-06]
 [4.6921517e-03 3.3467944e-04 3.2227337e-03 ... 2.0731888e-03
  1.3556154e-03 3.0760103e-04]
 [5.7905918e-01 1.1395101e-03 7.7154045e-03 ... 5.2771047e-03
  1.3814449e-01 1.6585598e-04]
 ...
 [2.9387014e-05 5.5083847e-01 9.4704889e-03 ... 8.3074541e-05
  6.5276829e-05 5.8108318e-04]
 [9.6382053e-07 1.7766629e-07 9.8776954e-06 ... 2.0847212e-08
  9.4356061e-08 6.3748342e-07]
 [2.5379664e-04 7.2680326e-05 7.8604020e-02 ... 1.0547791e-04
  1.4336433e-04 1.3985486e-04]]
